# <strong>Tensor Flow Deep Learning
<strong>Chapter 5 : Model Saving/ Tensor board</strong> 

<br>
## <strong>1 Modeling & Saving
<strong> 학습 모델을 저장 및 재사용
1. Tensor() : graph 구조를 결과로 출력한다

In [1]:
# 1. Csv에서 데이터 불러오기
import tensorflow as tf
import numpy as np

# unpack=True : 컬럼을 index 로 출력 
# array slice를 용이하기 위해서 설정을 활성화
data = np.loadtxt('./data/data.csv', delimiter = ',',
                  unpack = True, dtype = 'float32')
print(data.shape)
print('        털,  날개 | 기타, 포유류, 조류')
data.T 

/home/markbaum/Python/python36/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


(5, 6)
        털,  날개 | 기타, 포유류, 조류


array([[ 0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  1.,  0.],
       [ 1.,  1.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  1.]], dtype=float32)

In [2]:
# np.transpose() : unpack = True 를 되돌림
x_data = np.transpose(data[0:2])  # index 0,1    : 털, 날개          (element)
y_data = np.transpose(data[2:])   # index 2,3,4  : 기타, 포유류, 조류  (target)
print('x_data {}\ny_data {}'.format(x_data.shape, y_data.shape))

x_data (6, 2)
y_data (6, 3)


In [3]:
# 2. 신경망 모델 구성 
# 1) Tensor 를 정의
# trainable = False : 학습 횟수를 단순히 카운트
# 김석훈 강의코드는 for step .. 객체로 사용을 했지만, 
# 여기서는 '텐서보드'의 시각화를 위해 name='global_step'를 설정 
global_step = tf.Variable(0, trainable=False, name='global_step')
X = tf.placeholder(tf.float32,  [None, 2])
Y = tf.placeholder(tf.float32,  [None, 3])

# 2) Train 신경망 정의
# input: 2 --> hidden1 : 10 --> hidden2: 20 --> output: 3
W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

In [4]:
# 3) 최적화 모델의 cost 함수를 정의
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))
optimizer = tf.train.AdamOptimizer(learning_rate = 0.01)
# global_step로 넘겨준 변수를, 학습용 변수들을 최적화 할 때 마다 학습 횟수를 하나씩 증가
train_op = optimizer.minimize(cost, global_step = global_step)

In [5]:
# 3. 신경망 모델 학습 및 저장
# tf.train.Saver() : Train 객체를 저장
sess  = tf.Session()
saver = tf.train.Saver(tf.global_variables())
ckpt  = tf.train.get_checkpoint_state('./model')

# ./model 에 기존 학습한 모델이 있는지 확인 후 불러온다
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [6]:
# Train the Model
for step in range(2): 
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

# 최적화가 끝난 뒤, 변수를 저장
saver.save(sess, './data/dnn.ckpt', global_step=global_step)

Step: 1,  Cost: 0.870
Step: 2,  Cost: 0.852


'./data/dnn.ckpt-2'

In [7]:
# 4. 모델의 평가
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 2 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 83.33


In [8]:
tf.reset_default_graph()
sess.close()

<br>
## <strong>2 Tensor Board (학습과정을 시각화)
### <strong> tensor graph 설계
1. Tensor() : graph 구조를 결과로 출력한다

In [9]:
# 1. Load & Define the 'Data_set'
import tensorflow as tf
import numpy as np
data = np.loadtxt('./data/data.csv', delimiter=',',
                  unpack=True, dtype='float32')
print(data.shape)
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

global_step = tf.Variable(0, trainable=False, name='global_step')
X = tf.placeholder(tf.float32, [None, 2])
Y = tf.placeholder(tf.float32, [None, 3])

(5, 6)


In [10]:
# 2. 신경망 모델의 구성
# with tf.name_scope() : 묶인 블럭은, 텐서보드에서 개별 레이어로 표현
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)
    tf.summary.scalar('cost', cost)  # tf.summary.scalar : 수집하고 싶은 값을 지정

In [11]:
# 2. 신경망 모델을 학습
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())
ckpt = tf.train.get_checkpoint_state('./model')

if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [12]:
# 3. 텐서보드에서 표시해주기 위한 텐서들을 수집합니다.
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph) # 텐서를 저장할 디렉토리 설정

# tensorboard --logdir=./logs  # 웹서버를 실행
# http://localhost:6006

In [13]:
# 4. 최적화 진행
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    # 적절한 시점에 저장할 값들을 수집하고 저장
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))
    if step % 10 == 0:
        print('Step: %d, ' % sess.run(global_step),
              'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

saver.save(sess, './model/dnn.ckpt', global_step=global_step)

Step: 1,  Cost: 1.177
Step: 11,  Cost: 0.660
Step: 21,  Cost: 0.571
Step: 31,  Cost: 0.555
Step: 41,  Cost: 0.551
Step: 51,  Cost: 0.550
Step: 61,  Cost: 0.550
Step: 71,  Cost: 0.550
Step: 81,  Cost: 0.550
Step: 91,  Cost: 0.550


'./model/dnn.ckpt-100'

In [14]:
# 5. 결과 확인 (0: 기타 1: 포유류, 2: 조류)
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))
is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00
